 First, all the neccessary libraries need to be imported

In [2]:
import numpy as np
import cv2
import statsmodels.formula.api as sm
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Next, I am defining a couple functions that was provided by Udacity. I have modfied some of the code so that it has some of the default parameters defined in the function. By doing so, the parameters do not have to be explicity defined when the function is used.

In [3]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def canny(img, low_threshold=50, high_threshold=150):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


def gaussian_blur(img, kernel_size=3):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    # defining a blank mask to start with
    mask = np.zeros_like(img)

    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, np.int32([vertices]), ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def hough_lines(img, rho=2, theta=(np.pi/180) , threshold=30, min_line_len=50, max_line_gap=30):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.

    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                            maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).

    Think about things like separating line segments by their
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

Next, I defined a function that I wrote calleds **lines2fit**. This function essentially takes the lines data from Hough transform and coverts the data into greyscale. This makes the data easier to handle since each pixel is now only one integer (compared to three for a RGB image). Then, we take the grayscale data and find the pixel locations which has a value greater than 0. These pixel locations signfies where there is a line. Afterwards, the pixel locations are put through a linear regression to make a function that represents the line. Then, a function fit is used to determine the pixel closest to the bottom of the image and near the middle of the image. Lastly, the line on the image is drawn using the **drawlines** function from OpenCV.

In [4]:
def lines2fit(image,lines):

    final_gray = grayscale(lines)

    y_loc, x_loc = np.where(final_gray > 0)

    data = {"a": x_loc, "b":y_loc}
    model_lin = sm.ols(formula='a ~ b', data=data)
    result_lin = model_lin.fit()
    #print(result.summary())
    model_curve = sm.ols(formula='a ~ np.power(b, 2)', data=data)
    result_curve = model_curve.fit()

    mask_lines = np.zeros_like(image)

    y_bottom = image.shape[0]
    y_top = int(round(image.shape[0] * .6))



    x_pred_bottom = int(round((result_lin.params[1] * (y_bottom) + result_lin.params[0])))
    x_pred_top = int(round((result_lin.params[1] * (y_top) + result_lin.params[0])))

    #else:
    #    x_pred_bottom = int(round((result_curve.params[1] * np.power(y_bottom, 2) + result_curve.params[0])))
    #    x_pred_top = int(round((result_curve.params[1] * np.power(y_top, 2) + result_curve.params[0])))

    linemask = cv2.line(mask_lines, (x_pred_bottom, y_bottom), (x_pred_top, y_top), color=[255, 0, 0],
                    thickness=10)

    return linemask

I just the skills that we were taught through lesson 2 to create the next function **processimage**. The flow process is the following:
       1) Split the image into two region for each lane (left and right)
       2) Use the Gaussian Blur function
       3) Use the Canny Edge Detection Algorithm 
       4) Apply the Hough Tranform
       5) Mask the region into two using the regions defined in step 1 
       6) Use the lines2fit function in the previous cell
       7) Use the weighted_img to overlay the modified result and original image

In [5]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    gray = grayscale(image)
    x_start_point = .14
    y_start_point = 1

    x_offset_percent = .42
    y_offset_percent = 0.64
    center_offset = 0.05

    left_bottom = [int(round(x_start_point * image.shape[1])), int(round(y_start_point * image.shape[0]))]
    left_top = [int(round(x_offset_percent * image.shape[1])), int(round(y_offset_percent * image.shape[0]))]
    center_top = [int(round((.5 + center_offset) * image.shape[1])), int(round(y_offset_percent * image.shape[0]))]
    center_bottom = [int(round((.5 + center_offset) * image.shape[1])), int(round(y_start_point * image.shape[0]))]

    right_bottom = [int(round((1 - x_start_point + center_offset) * image.shape[1])),
                    int(round(y_start_point * image.shape[0]))]
    right_top = [int(round((1 - x_offset_percent + center_offset) * image.shape[1])),
                 int(round(y_offset_percent * image.shape[0]))]

    left_vertices = np.array([left_bottom, left_top, center_top, center_bottom], np.int32)
    right_vertices = np.array([right_bottom, right_top, center_top, center_bottom], np.int32)

    blur_gray = gaussian_blur(gray)

    edges = canny(blur_gray)

    lines = hough_lines(edges)

    left_lines = region_of_interest(lines, left_vertices)
    right_lines = region_of_interest(lines, right_vertices)



    left_mask = lines2fit(image,left_lines)
    right_mask = lines2fit(image,right_lines)

    pre_image = weighted_img(left_mask, image)
    final_image = weighted_img(right_mask, pre_image)
    #plt.imshow(final_image)
    #plt.show()

    return final_image

Last but not least, the video file is pipelined into the processimage function to create the final video with the lane detection algorithm

In [6]:
white_output = 'output1.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)


yellow_output = 'output2.mp4'
clip2 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video output1.mp4
[MoviePy] Writing video output1.mp4


100%|█████████▉| 221/222 [00:48<00:00,  4.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output1.mp4 

[MoviePy] >>>> Building video output2.mp4
[MoviePy] Writing video output2.mp4


100%|█████████▉| 681/682 [02:38<00:00,  3.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output2.mp4 



In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))







In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

# Reflections

The biggest challenge for me in this project was to brush up on my Python 3. I have always been coding in Python 2 so I figured it was a good time to transition to Python 3. Also, for some reason finding the location of the pixel where for the regression took me longer than expected. I originally tried using the RGB array to find the pixel location but splitting the array up into only into a 2D array took too much work so I resorted to the grayscale function. Also, I was too used to matlab and I declared the function to a single variable and I expected to be able to split the array up conveniently but it didn't work.

I tried to fit the lines to the function ax^2+ b and ax+b and compared either the r-squared results or AIC/BIC to see which model fit better. Unfortunately the results were really bad. Ideally, I would fit it to ax^2 +bx + c but it would overfit the points. Ideally, I would like to try to fit to a couple order of polynomials and use a k-crossfold to see which one fits the best without overfitting. I would love to try this out if I get more time to work on this project.

The negative aspect of my program is that the side with the sectioned lines tend to be shaky. I adjusted my values according to the suggestion to my original project review and the results ended much better! To get the best results, I would build some sort of LSTM regression or use a moving average on the input value of the regression to get better results. Side note, I love hearing critism (positive or negative!) so don't worry about writing too much! I would enjoy reading all of it! I really enjoyed the link you provided me for better coding practices. I picked up way too many bad coding practices since I was a teenager and I haven't really code enough to get rid of them (I"m a mechanical engineer so I haven't got much chance to code professionally). I think these next 9 months would be a great opportunity to become a better coder.

Overall, it was really fun to learn about the OpenCV API and implement it on this project. It was really fun and satisfying to see the final results to work! I plan on learning more about computer vision on the side and see if I can do any additional fun side projects. I heard that Udacity might setup something so that students can get paid for side-work and I would love to join to learn more! 

